# Spiegabilità: LORE - Estrazione ed elaborazione regole

In [ ]:
# Caricamento dei dati di addestramento
train = pd.read_csv(f"XAI_Datasets/XAI_LORE_DoubleFeature.tsv", sep='\t')  

# Salvataggio delle classi e del nome della colonna delle label in variabili
y_train = np.array(train['Order_Label'])
class_name = 'Order_Label'

# Preparazione del set di addestramento per LORE 
res = prepare_dataset(train, class_name)
X, feature_names, class_values, numeric_columns, rdf, real_feature_names, features_map = res

# Preparazione dei dati di train e test
y = np.array(X['Order_Label'])
X = X.drop(labels=["Order_Label"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.3)

# Definizione ed addestramento del modello Random Forest
bb = RandomForestClassifier()
bb.fit(X_train, y_train)

# Funzione per effettuare predizioni delle classi 
def bb_predict(X):
    return bb.predict(X)
# Funzione per estrarre le probabilità predette associate alle classi di output
def bb_predict_proba(X):
    return bb.predict_proba(X)

# Valutazione dell'accuratezza 
y_pred = bb_predict(X_test)
# Stampa del classification report del Random Forest
print('Accuracy %.3f' % accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Calcolo del numero totale di campioni nel dataset
n_samples = len(rdf)
test_size = 0.3
# Calcolo il numero di campioni nel training set in base alla dimensione del test set
n_train = int(n_samples * (1 - test_size))
random_state = 0
# Crea un array di indici casuali per il training set utilizzando il seme specificato
train_indices = np.random.RandomState(random_state).choice(n_samples, size=n_train, replace=False)
# Seleziona le colonne desiderate dal dataset di training e ottiene i relativi valori 
K = rdf.iloc[train_indices][real_feature_names].values

# Istanziazione di LORE per spiegare le istanze individuali
lore_explainer = LOREM(K, bb_predict, feature_names, class_name, class_values, numeric_columns, features_map,
                       neigh_type='geneticp', categorical_use_prob=True, continuous_fun_estimation=False,
                       size=1000, ocr=0.1, random_state=42, ngen=10, bb_predict_proba=bb_predict_proba,
                       verbose=True)

# Definizione di una lista di indici per selezionare le istanze da spiegare (le prime 565)
i_list = [i for i in range(565)]
# Creazione di una lista di tuple (x, i) che rappresentano le istanze da spiegare
x_list = []
for i in i_list:
    x_list.append((X_train.iloc[i], i))
    
# Definizione lista per memorizzare le spiegazioni generate
exp_list_doubleFeature = []
# Genera spiegazioni per ogni tupla (x, i) presente in x_list
for x, i in x_list:
    x_array = x.to_numpy()
    # Genera una spiegazione utilizzando LORE
    exp = lore_explainer.explain_instance(x_array, samples=300, use_weights=True, metric=neuclidean)
    # Aggiunge la spiegazione all'elenco exp_list insieme all'indice corrispondente
    exp_list_doubleFeature.append((exp, i))
 

 #Lista vuota per memorizzare le regole estratte dalle spiegazioni
exp_list_rules=[]
# Iterare attraverso la lista di tuple (exp, i)
for exp, i in exp_list_doubleFeature:

    # Stampa l'indice del record con annessa spiegazione
    print("Explanation per il record: " + str(i)) 
    print(exp)
    # Aggiungo la regola estratta dalla spiegazione alla lista
    exp_list_rules.append(str(exp.rule))
    
# Per salvare in un file rules generate 
#with open('LORE_Rules_DoubleFeature.json', 'w') as file:
    #json.dump(exp_list_rules, file)
# Per leggere da file le rules generate 
#with open('Lore_Rules_Json/LORE_Rules_DoubleFeature.json', 'r') as file:
    #exp_list_rules = json.load(file)
    
# Funzione per estrarre gli intervalli dei valori delle feature 
def extract_feature_intervals(string):
    
    features = []
    # Sostituisce '+' con '__' nella stringa per evitare errori di parsing
    string = string.replace('+', '__')
    # Estrae le informazioni sulle feature e la label dalla stringa
    match = re.match(r'\{(.+?)\} --> \{ Order_Label: (\d) \}', string)
    features_str, label = match.groups()
    label = int(label)
    
    # Definizione del pattern per estrarre le informazioni di ogni feature dalla stringa
    pattern = r'(.+?)\s*([<>]=?)\s*(-?\d+\.\d+)(?:,|$)'
    # Trova tutte le corrispondenze dei pattern nella stringa di feature
    feature_matches = re.findall(pattern, features_str)
    
    # Itera attraverso le corrispondenze delle feature
    for match in feature_matches:
        # Salva nelle variabili i nomi delle features, degli operatori di comparazione e il valore a destra del segno
        feature_name = match[0].strip()
        comparison_operator = match[1].strip()
        number = match[2].strip()
        
        # Definisce l'intervallo come stringa contenente segno e valore
        interval = f'{comparison_operator} {number}'
        # Aggiunge le informazioni sulla feature e la label alla lista
        features.append((feature_name, interval, label))
        
    return features

# Funzione per calcolare quante volte le feature compaiono in ogni rule a seconda della classe
def count_feature_occurrences(data):
    
    # Dizionario annidato per contare quante volte compare ciascuna feature per ogni label
    feature_count = defaultdict(lambda: defaultdict(int))
    
    # Itera attraverso ogni stringa nel dataset
    for string in data:
        # Estrae gli intervalli delle feature dalla stringa usando la funzione extract_feature_intervals
        feature_intervals = extract_feature_intervals(string)
        # Itera attraverso ogni feature_interval estratto
        for feature, interval, label in feature_intervals:
            # Incrementa il conteggio della feature per la label corrispondente
            feature_count[label][feature] += 1
    
    # Ordina i risultati in modo decrescente per count per ogni label
    sorted_feature_count = {label: dict(sorted(features.items(), key=lambda x: x[1], reverse=True)) for label, features in feature_count.items()}

    # Mostra le frequenze delle features in ordine decrescente per ogni classe
    for label, features in sorted_feature_count.items():
        print(f"Classifica per la label {label}:")
        for feature, count in features.items():
            print(f"{feature}: {count} volte")
        print("\n")

    return sorted_feature_count

# Funzione per aggregare gli intervalli a seconda della feature e della label
def aggregate_intervals(data):
    
    # Dizionario annidato per aggregare gli intervalli delle feature per ogni label
    aggregated_intervals = defaultdict(lambda: defaultdict(list))
    
    # Itera attraverso ogni stringa nel dataset
    for string in data:
        # Estrae gli intervalli delle feature dalla stringa usando la funzione extract_feature_intervals
        feature_intervals = extract_feature_intervals(string)
        # Itera attraverso ogni feature_interval estratto
        for feature, interval, label in feature_intervals:
            # Aggiunge l'intervallo alla lista delle feature corrispondenti per la label
            aggregated_intervals[label][feature].append(interval)
    
    # Per ogni label e feature in aggregated_intervals
    for label, features in aggregated_intervals.items():  
        print(f"Intervalli per label {label}:")
        for feature, intervals in features.items():
            # Chiamata alla funzione merge_intervals per unire gli intervalli sovrapposti
            result_interval = merge_intervals(intervals)
            print(f"{feature}: {result_interval}")
        print("\n")
        
    return aggregated_intervals

# Funzione per ottenere un unione degli intervalli in casi di sovrapposizione
def merge_intervals(intervals):
    # Inizializza lower e upper bounds con valori di default
    lower_bound = float('-inf')
    upper_bound = float('inf')

    # Flag per verificare se ci sono operatori '<' o '<='
    has_less_than = False 

    # Flag per verificare se ci sono operatori '>' o '>='
    has_greater_than = False

    # Lista per memorizzare i valori minimi e massimi
    min_values = []
    max_values = []

    # Itera sugli intervalli e aggiorna i valori di lower e upper bounds
    for interval in intervals:
        # Separa l'operatore e il valore dall'intervallo
        op, val = interval.split()
        val = float(val)
        
        # Aggiorna le liste dei valori minimi e massimi in base all'operatore   
        if op == '<':
            # Imposta il flag per indicare la presenza di '<'
            has_less_than = True 
            # Aggiunge il valore massimo alla lista
            max_values.append(val)
        
        elif op == '<=':
            # Imposta il flag per indicare la presenza di '<='
            has_less_than = True
            # Aggiunge il valore massimo alla lista
            max_values.append(val)
            
        elif op == '>':
            # Imposta il flag per indicare la presenza di '>'
            has_greater_than = True
            # Aggiunge il valore minimo alla lista
            min_values.append(val)
            
        elif op == '>=':
            # Imposta il flag per indicare la presenza di '>='
            has_greater_than = True
            # Aggiunge il valore minimo alla lista
            min_values.append(val)

    # Se ci sono solo operatori '<' o '<='
    if has_less_than and not has_greater_than:
        # Restituisce l'intervallo da '-∞' al massimo valore
        result_interval = f'(-∞, {max(max_values)}]'
        
    # Se ci sono solo operatori '>' o '>='
    elif has_greater_than and not has_less_than:
        # Restituisce l'intervallo dal minimo valore a '+∞'
        result_interval = f'[{min(min_values)}, ∞)'
        
    # Se ci sono entrambi gli operatori, calcola l'intervallo risultante
    else:
        # Imposta il limite inferiore
        lower_bound = min(min_values) 
        # Imposta il limite superiore
        upper_bound = max(max_values) 

        # Se il limite inferiore è minore o uguale al limite superiore
        if lower_bound <= upper_bound:
            # Crea la stringa rappresentante l'intervallo
            result_interval = f'(-∞,∞), valore minore: {lower_bound}, valore maggiore: {upper_bound}'
        else:
            result_interval = 'Intervallo vuoto'

    # Restituisce la stringa intervallo
    return result_interval


# Conta quante volte compare ciascuna feature per ogni label
feature_count = count_feature_occurrences(exp_list_rules)
# Aggrega gli intervalli per ciascuna feature
aggregated_intervals = aggregate_intervals(exp_list_rules)